# Swaption European swaption pricing and sensitivities 

This notebook demonstrates how to access and use the **Swaption** functionalities within the **LSEG Financial Analytics SDK**. 

Swaption instruments are interest rate derivatives that provide protection against adverse interest rate movements. The following swaption types are supported:
- Payer Swaption: Right to enter a swap as the fixed-rate payer (protection against rising rates)
- Receiver Swaption: Right to enter a swap as the fixed-rate receiver (protection against falling rates)

With following exercise styles:
- European Style: Exercisable at expiration date
- American Style: Exercisable at any date before expiration
- Bermudan Style: Exercisable on multiple specified dates

**You will be able to:**
- Define European Swaption instruments with advanced parameters
- Retrieve and evaluate sensitives, implied volatility and valuation characteristics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.swaption` - for Swaption instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import swaption  as sw

import pandas as pd
import json
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Swaption instrument you need to follow a structured 3-step process:

1. **Swaption Definition** - Specify basic swaption parameters (strike, expiry, underlying swap details)
2. **Swaption Instrument Definition** - Create the instrument object for pricing
3. **Pricing Preferences** - Configure pricing parameters and market data settings, optional

We give in this notebook three European Swaption Examples:

 - **Example 1: 6M x 5Y USD Payer** - Corporate hedging strategy using physical settlement and SOFR underlying swap

 - **Example 2: 1Y x 10Y GBP Receiver** - Portfolio protection with cash settlement and SONIA underlying swap  

 - **Example 3: 3M x 2Y EUR Payer (SELL)** - Premium income strategy with customed ESTR swap leg definitions

Each example demonstrates different European exercise characteristics, settlement methods, and underlying swap configurations.

### Swaption Definition

#### Example 1: 6-Month Into 5-Year USD Payer Swaption

In [2]:

print("Example 1: 6M x 5Y USD Payer European Swaption")
print("-" * 50)

# Step 1: European Payer Swaption Definition
euro_payer_definition = sw.SwaptionDefinition(
    # European Exercise - Key differentiator
    exercise_style=sw.IPAExerciseStyleEnum.EURO,               # European: exercise ONLY at expiration. Options: EURO, AMER, BERM
    
    # Core Swaption Parameters
    buy_sell=sw.IPABuySellEnum.BUY,                            # Buy protection against rising rates. Options: BUY, SELL
    swaption_type=sw.SwaptionTypeEnum.PAYER,                   # Payer: right to PAY fixed in underlying swap. Options: PAYER, RECEIVER
    
    # Option Timing - 6M into 5Y
    start_date=dt.datetime(2025, 1, 20),                       # Option effective date (T+2 from trade)
    end_date=dt.datetime(2025, 7, 21),                         # Option expiry (ONLY exercise date for European)
    
    # Financial Terms
    notional_amount=50_000_000,                                # $50M notional for corporate hedge
    strike_percent=4.25,                                       # 4.25% strike (fixed rate in underlying swap)
    
    # Settlement Configuration  
    settlement_type=sw.SettlementTypeEnum.PHYSICAL,           # Physical: actually enter swap if exercised. Options: PHYSICAL, CASH, CCP
    premium_settlement_type=sw.PremiumSettlementTypeEnum.SPOT, # Premium paid at trade date. Options: SPOT, FORWARD, SCHEDULE
    delivery_date=dt.datetime(2025, 7, 23),                   # Underlying swap start date (T+2 from expiry)
    
    # European Swaption Specific Parameters
    instrument_tag="EURO_6Mx5Y_PAYER_USD",                    # Tag for identification
    spread_vs_atm_in_bp=0.0,                                  # At-the-money strike (0bp spread from ATM)
    
    # Underlying 5-Year USD SOFR Swap
    underlying_definition=sw.SwapDefinition(
        template="OIS_SOFR",                                   # Standard USD SOFR overnight index swap
        start_date=dt.datetime(2025, 7, 23),                   # Swap effective date (delivery_date)
        end_date=dt.datetime(2030, 7, 23),                     # 5Y maturity from swap start in date format
        instrument_tag="5Y_SOFR_Underlying_Swap"
    )
)

print(f"✓ European Payer Swaption Created:")
print(f"  Swaption Type: {euro_payer_definition.swaption_type} (Right to pay fixed)")
print(f"  Exercise Style: {euro_payer_definition.exercise_style} (Only one exercise opportunity at expiration date)")
print(f"  Position: {euro_payer_definition.buy_sell} (buy protection against rising rates)")
print(f"  Option Period: {euro_payer_definition.start_date.strftime('%Y-%m-%d')} to {euro_payer_definition.end_date.strftime('%Y-%m-%d')} (6M)")
print(f"  Underlying: {(euro_payer_definition.end_date - euro_payer_definition.start_date).days} days into 5Y swap")
print(f"  Strike Rate: {euro_payer_definition.strike_percent}% (fixed rate to pay if exercised)")
print(f"  Settlement: {euro_payer_definition.settlement_type}")
print(f"  ATM Spread: {euro_payer_definition.spread_vs_atm_in_bp}bp (At-the-money)")
print(f"  Premium Settlement: {euro_payer_definition.premium_settlement_type}")

Example 1: 6M x 5Y USD Payer European Swaption
--------------------------------------------------
✓ European Payer Swaption Created:
  Swaption Type: SwaptionTypeEnum.PAYER (Right to pay fixed)
  Exercise Style: IPAExerciseStyleEnum.EURO (Only one exercise opportunity at expiration date)
  Position: IPABuySellEnum.BUY (buy protection against rising rates)
  Option Period: 2025-01-20 to 2025-07-21 (6M)
  Underlying: 182 days into 5Y swap
  Strike Rate: 4.25% (fixed rate to pay if exercised)
  Settlement: SettlementTypeEnum.PHYSICAL
  ATM Spread: 0.0bp (At-the-money)
  Premium Settlement: PremiumSettlementTypeEnum.SPOT


### Example 2: 1-Year Into 10-Year GBP Receiver Swaption

In [3]:
print("Example 2: 1Y x 10Y GBP Receiver European Swaption")
print("-" * 50)

# Step 2: European Receiver Swaption Definition  
euro_receiver_definition = sw.SwaptionDefinition(
    # European Exercise Style
    exercise_style=sw.IPAExerciseStyleEnum.EURO,               # European style - single exercise date
    
    # Receiver Swaption Parameters
    buy_sell=sw.IPABuySellEnum.BUY,                            # Buy protection against falling rates
    swaption_type=sw.SwaptionTypeEnum.RECEIVER,                # Receiver: right to RECEIVE fixed in underlying swap
    
    # Extended Option Period
    start_date=dt.datetime(2025, 1, 20),                       # Option start
    end_date=dt.datetime(2026, 1, 20),                         # 1-year option expiry in date format
    
    # Larger Notional
    notional_amount=100_000_000,                               # £100M notional
    strike_percent=3.75,                                       # 3.75% receiver strike
    
    # Cash Settlement Configuration
    settlement_type=sw.SettlementTypeEnum.CASH,               # Cash settlement: receive PnL without entering swap
    premium_settlement_type=sw.PremiumSettlementTypeEnum.SPOT, # Upfront premium payment
    delivery_date=dt.datetime(2026, 1, 22),                   # Cash settlement calculation date
    
    # European Swaption Specific Parameters
    instrument_tag="EURO_1Yx10Y_RECEIVER_GBP",
    spread_vs_atm_in_bp=-25.0,                                # 25bp below ATM (out-of-the-money receiver)

    # Underlying 10-Year GBP SONIA Swap
    underlying_definition=sw.SwapDefinition(
        template="OIS_SONIA", 
        start_date=dt.datetime(2026, 1, 22),                   # 10Y swap starts at option expiry
        end_date=dt.datetime(2036, 1, 22),                     # 10Y tenor in date format
        instrument_tag="10Y_SONIA_Underlying_Swap"
    )
)

print(f"✓ European Receiver Swaption Created:")
print(f"  Swaption Type: {euro_receiver_definition.swaption_type} (Right to receive fixed)")
print(f"  Exercise Style: {euro_receiver_definition.exercise_style} (Only one exercise opportunity at expiration date)")
print(f"  Position: {euro_receiver_definition.buy_sell} (buy protection against falling rates)")
print(f"  Option Period: {euro_receiver_definition.start_date.strftime('%Y-%m-%d')} to {euro_receiver_definition.end_date.strftime('%Y-%m-%d')} (1Y)")
print(f"  Underlying: 10Y SONIA swap")
print(f"  Strike rate: {euro_receiver_definition.strike_percent}% (fixed rate to receive if exercised)")
print(f"  Settlement: {euro_receiver_definition.settlement_type} (no physical swap delivery)")
print(f"  ATM Spread: {euro_receiver_definition.spread_vs_atm_in_bp}bp (Out-of-the-money)")
print(f"  Premium Settlement: {euro_receiver_definition.premium_settlement_type}")

Example 2: 1Y x 10Y GBP Receiver European Swaption
--------------------------------------------------
✓ European Receiver Swaption Created:
  Swaption Type: SwaptionTypeEnum.RECEIVER (Right to receive fixed)
  Exercise Style: IPAExerciseStyleEnum.EURO (Only one exercise opportunity at expiration date)
  Position: IPABuySellEnum.BUY (buy protection against falling rates)
  Option Period: 2025-01-20 to 2026-01-20 (1Y)
  Underlying: 10Y SONIA swap
  Strike rate: 3.75% (fixed rate to receive if exercised)
  Settlement: SettlementTypeEnum.CASH (no physical swap delivery)
  ATM Spread: -25.0bp (Out-of-the-money)
  Premium Settlement: PremiumSettlementTypeEnum.SPOT


#### Example 3: Short Dated European Swaption with Custom Underlying Definition

In [4]:
print("Example 3: 3M x 2Y EUR Payer European Swaption")
print("-" * 50)

# Step 3: Short-dated European Swaption (Different Currency)
euro_short_definition = sw.SwaptionDefinition(
    exercise_style=sw.IPAExerciseStyleEnum.EURO,               # European exercise
    buy_sell=sw.IPABuySellEnum.SELL,                           # Sell swaption (receive premium)
    swaption_type=sw.SwaptionTypeEnum.PAYER,                   # Payer swaption: right to pay fixed in underlying swap for buyer.
    
    # Short Option Period
    start_date=dt.datetime(2025, 1, 20),
    end_date=dt.datetime(2025, 4, 21),                         # 3-month option in date format
    
    # EUR Denominated
    notional_amount=25_000_000,                                # €25M notional
    strike_percent=3.00,                                       # 3.00% EUR strike

    # Cash Settlement Configuration
    settlement_type=sw.SettlementTypeEnum.CASH,                # Cash settlement
    premium_settlement_type=sw.PremiumSettlementTypeEnum.FORWARD, # Forward premium settlement
    delivery_date=dt.datetime(2025, 4, 23),
    
    # Short-dated Strategy
    instrument_tag="EURO_3Mx2Y_PAYER_EUR_SELL",
    spread_vs_atm_in_bp=15.0,                                 # 15bp above ATM (in-the-money payer)
    
    # EUR 2Y ESTR Swap - Custom Definition 
    underlying_definition=sw.SwapDefinition(
        start_date=dt.datetime(2025, 4, 23),                   # Swap effective date
        end_date=dt.datetime(2027, 4, 23),                     # 2Y swap maturity in date format
        instrument_tag="2Y_ESTR_Full_Custom_Swap",
        
        legs=[
            # Fixed Rate Leg (Payer Leg) - What swaption holder pays if exercised
            sw.SwapLegDefinition(
                direction=sw.IPADirectionEnum.PAID,                     
                notional_amount=25_000_000,                            # €25M notional
                notional_ccy="EUR",                                   
                interest_type=sw.InterestTypeEnum.FIXED,               
                fixed_rate_percent=3.00,                               # 3.00% fixed rate (matches strike)
                interest_payment_frequency=sw.InterestPaymentFrequencyEnum.ANNUAL,  # Annual payments
                interest_calculation_method=sw.InterestCalculationMethodEnum.DCB_30_360,  # 30/360 day count
                payment_business_day_convention=sw.PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING, # Modified following payment business day convention 

                leg_tag="FixedLeg",                                 
 
            ),
            
            # Floating Rate Leg (Receiver Leg) - What swaption holder receives if exercised
            sw.SwapLegDefinition(
                direction=sw.IPADirectionEnum.RECEIVED,               
                notional_amount=25_000_000,                            # €25M notional (same as fixed leg)
                notional_ccy="EUR",                                  
                interest_type=sw.InterestTypeEnum.FLOAT,             
                
                # ESTR (Euro Short Term Rate) Index Configuration
                index_name="ESTR",                                     # European Short Term Rate
                index_tenor="1D",                                      # Daily rate (overnight)
                interest_payment_frequency=sw.InterestPaymentFrequencyEnum.ANNUAL,  # Annual payments
                index_reset_frequency=sw.IndexResetFrequencyEnum.EVERY_WORKING_DAY, 
                
                # EUR Market Conventions
                interest_calculation_method=sw.InterestCalculationMethodEnum.DCB_ACTUAL_360,  # ACT/360 for ESTR
                payment_business_day_convention=sw.PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING, # Modified following payment business day convention 
                
                # Date Settings  
                leg_tag="FloatingLeg",                                

            )
        ]
    )
)


print(f"✓ Short European Swaption with Full Swap Definition Created:")
print(f"  Swaption Type: {euro_short_definition.swaption_type} (Right to pay fixed)")
print(f"  Exercise Style: {euro_short_definition.exercise_style} (Only one exercise opportunity at expiration date)")
print(f"  Position: {euro_short_definition.buy_sell} (premium received)")
print(f"  Option Period: {euro_short_definition.start_date.strftime('%Y-%m-%d')} to {euro_short_definition.end_date.strftime('%Y-%m-%d')} (3M)")
print(f"  Underlying: customed 2Y ESTR Swap (see details below)")
print(f"  Settlement: {euro_short_definition.settlement_type}")
print(f"  ATM Spread: {euro_short_definition.spread_vs_atm_in_bp}bp (In-the-money)")
print(f"  Premium Settlement: {euro_short_definition.premium_settlement_type}")
print()
print(f"  Underlying Swap Details:")
print(f"    Fixed Leg: PAY {euro_short_definition.underlying_definition.legs[0].fixed_rate_percent}% annually")
print(f"    Floating Leg: RECEIVE ESTR compounded annually")
print(f"    Day Count: Fixed=30/360, Floating=ACT/360")
print(f"    Payment Frequency: {euro_short_definition.underlying_definition.legs[0].interest_payment_frequency}")
print(f"    Business Day Convention: {euro_short_definition.underlying_definition.legs[0].payment_business_day_convention}")

Example 3: 3M x 2Y EUR Payer European Swaption
--------------------------------------------------
✓ Short European Swaption with Full Swap Definition Created:
  Swaption Type: SwaptionTypeEnum.PAYER (Right to pay fixed)
  Exercise Style: IPAExerciseStyleEnum.EURO (Only one exercise opportunity at expiration date)
  Position: IPABuySellEnum.SELL (premium received)
  Option Period: 2025-01-20 to 2025-04-21 (3M)
  Underlying: customed 2Y ESTR Swap (see details below)
  Settlement: SettlementTypeEnum.CASH
  ATM Spread: 15.0bp (In-the-money)
  Premium Settlement: PremiumSettlementTypeEnum.FORWARD

  Underlying Swap Details:
    Fixed Leg: PAY 3.0% annually
    Floating Leg: RECEIVE ESTR compounded annually
    Day Count: Fixed=30/360, Floating=ACT/360
    Payment Frequency: InterestPaymentFrequencyEnum.ANNUAL
    Business Day Convention: PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING


### Swaption Instrument Definition

We create for the each of the three examples a swaption instrument to be priced in one batch.

In [5]:
print("Creating Instrument Objects for Batch Pricing...")
print("-" * 50)

# Create instrument containers
euro_payer_instrument = sw.SwaptionDefinitionInstrument(definition=euro_payer_definition)
euro_receiver_instrument = sw.SwaptionDefinitionInstrument(definition=euro_receiver_definition) 
euro_short_instrument = sw.SwaptionDefinitionInstrument(definition=euro_short_definition)

# Batch instrument list for simultaneous pricing
european_swaptions_batch = [euro_payer_instrument, euro_receiver_instrument, euro_short_instrument]

print(f"✓ Created {len(european_swaptions_batch)} European swaption instruments")
print("  - 6M x 5Y USD Payer (Physical settlement)")
print("  - 1Y x 10Y GBP Receiver (Cash settlement)")
print("  - 3M x 2Y EUR Payer Sold (Cash settlement)")
print()

Creating Instrument Objects for Batch Pricing...
--------------------------------------------------
✓ Created 3 European swaption instruments
  - 6M x 5Y USD Payer (Physical settlement)
  - 1Y x 10Y GBP Receiver (Cash settlement)
  - 3M x 2Y EUR Payer Sold (Cash settlement)



### Pricing Preferences

#### Setup pricing parameters

In [6]:
print("Configuring European Swaption Pricing Parameters...")
print("-" * 50)

# Pricing parameters optimized for European swaption analytics
euro_pricing_params = sw.SwaptionPricingParameters(
    valuation_date=dt.datetime(2025, 1, 18),                   # Current valuation date
    price_side=sw.PriceSideEnum.MID,                           # Mid-market pricing for fair value
    report_ccy="USD"                                           # Optional: force USD reporting
)

print(f"✓ European Swaption Pricing Configuration:")
print(f"  Valuation Date: {euro_pricing_params.valuation_date.strftime('%Y-%m-%d')}")
print(f"  Price Side: {euro_pricing_params.price_side}")
print(f"  Report Currency: {euro_pricing_params.report_ccy}")
print()

Configuring European Swaption Pricing Parameters...
--------------------------------------------------
✓ European Swaption Pricing Configuration:
  Valuation Date: 2025-01-18
  Price Side: PriceSideEnum.MID
  Report Currency: USD



#### European Swaption Fields Setup

We select the below the following fields that are interesting for European Swaption pricing:

- **InstrumentTag**: Tag for identification
- **Notional currency**: Currency of the swaption
- **Market Value**: Pure option value without early exercise premium
- **Report currency**: Currency of swaption valuation and metrics
- **Premium**: Market value divided by notional. Lower than American equivalents due to single exercise date.
- **Implied volatility**: Equivalent Black-Scholes (resp. Bachelier) implied volatility if expressed in percentage (resp. in bps) 
- **Delta (Δ)**:  Change in the swaption market value caused by 1bp change in the swap rate
- **Gamma (Γ)**: Change in the swaption delta caused by 1bp change in the swap rate
- **Vega (ν)**: Change in the swaption market value per 1bp change in the normal volatility (if market data has only log normal volatility surface, it is 1% change in the lognormal volatility) 
- **Theta (Θ)**: Change in the swaption market value per one-day decrease in its time to expiration
- **Hedge Notional**: The notional amount of the underlying swap that allows to hedge the swaption.

In [7]:
basic_fields="InstrumentTag, NotionalCcy, MarketValueInDealCcy, ReportCcy, MarketValueInReportCcy, premiumPercent, premiumBp, ImpliedVolatilityPercent, ImpliedVolatilityBp"
sensitivity_fields_percent="deltaPercent, gammaPercent, vegaPercent, thetaPercent"
sensitivity_fields_absolute="DeltaAmountInReportCcy, GammaAmountInReportCcy, VegaAmountInReportCcy, ThetaAmountInReportCcy, HedgeNotionalAmountInReportCcy"

# combine basic and sensitivity fields
fields=basic_fields + "," + sensitivity_fields_percent + "," + sensitivity_fields_absolute

## Request Execution

In [8]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instrument definitions for batch processing

try:
    response = sw.price(
        definitions=european_swaptions_batch,
        pricing_preferences=euro_pricing_params,
        fields=fields
    )
    
    all_calculations_ok = True
    for item in response['data']['analytics']:
        analytics_data = item
        if analytics_data['error'] == {}:
            valuation_data = analytics_data.get('valuation', {})
        else:
            all_calculations_ok = False
            print("   - InstrumentTag:", analytics_data['tabularData']['data'][0])
            print(f"   Pricing error: {analytics_data['error']}")
    if all_calculations_ok:
        print("Calculation successful!")
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise


Calculation successful!


## Results Display

#### Key Sections in the European Swaption `response` JSON

 - **definitions**: Instrument setup (e.g., strike rate, expiry dates, notional, underlying swap structure), using the SwaptionDefinition that we created.

 - **pricingPreferences**: Valuation date, price side (BID/ASK/MID/LAST), currency settings.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`)
   - **description**: Instrument summary SwaptionDefinition and default fields used in the calculation
   - **greeks**: Main risk metrics calculated on swaption instrument (`deltaPercent`, `deltaAmountInReportCcy`, `gammaPercent`, `gammaAmountInReportCcy`, `vegaPercent`, `vegaAmountInReportCcy`, `thetaPercent`, `thetaAmountInReportCcy`)
   - **pricingAnalysis**: Further information on pricing with the below fields:
      - `ImpliedVolatilityPercent`: Black-Scholes market-implied volatility for European exercise
      - `ImpliedVolatilityBps`: Bachelier market-implied volatility for European exercise
   - **valuation**: Detailed valuation of the swaption instrument with the below fields:
      - `MarketValueInDealCcy`: Present value of the swaption in deal currency
      - `MarketValueInReportCcy`: Present value of the swaption in report currency
      - `PremiumPercent`: Swaption premium as percentage of notional
      - `PremiumBp`: Swaption premium in basis points
   - **error**: Root cause of any issues, if encountered

We give below for the first deal the valuation and greeks details, as well as for the three deals the tabular data details, that include information on valuation, implied volatility and greeks .


In [9]:
# Access the valuation object for #1 deal
valuation = response.data.analytics[0].valuation
print("Swaption Valuation Results:")
print(json.dumps(valuation.as_dict(), indent=4))

# Access the greeks object for #1 deal
print("Swaption Greeks Results:")
print(json.dumps(response.data.analytics[0].greeks.as_dict(), indent=4))

Swaption Valuation Results:
{
    "marketValueInDealCcy": 515536.72614804,
    "marketValueInReportCcy": 515536.72614804,
    "premiumPercent": 1.03107345229608,
    "premiumBp": 103.107345229608
}
Swaption Greeks Results:
{
    "deltaAmountInReportCcy": 9390.29425312561,
    "deltaPercent": 1.87805885062512,
    "gammaAmountInReportCcy": 115.412031413081,
    "gammaPercent": 230.824062826161,
    "thetaAmountInReportCcy": -1694.3053497096,
    "thetaPercent": -3.38861069941919e-05,
    "vegaAmountInReportCcy": 6123.76079716225,
    "vegaPercent": 1.22475215943245
}


In [10]:
# Convert swaption analytics to DataFrame for better visualization
print("Swaption Analytics Summary:")
columns = [item['name'] for item in response['data']['analytics'][0]['tabularData']['headers']]
response_data = [item['tabularData']['data'] for item in response['data']['analytics']]

response_df = pd.DataFrame(response_data, columns=columns)

# Set InstrumentTag as index for better readability
response_df.set_index('InstrumentTag', inplace=True)

# Round numerical values to 2 decimal places while keeping strings unchanged
response_df_rounded = response_df.copy()
for col in response_df_rounded.columns:
    if response_df_rounded[col].dtype in ['float64', 'int64']:
        response_df_rounded[col] = response_df_rounded[col].round(2)

display(response_df_rounded.T)

Swaption Analytics Summary:


InstrumentTag,EURO_6Mx5Y_PAYER_USD,EURO_1Yx10Y_RECEIVER_GBP,EURO_3Mx2Y_PAYER_EUR_SELL
NotionalCcy,USD,GBP,EUR
MarketValueInDealCcy,515536.73,1811261.97,-2376.59
ReportCcy,USD,USD,USD
MarketValueInReportCcy,515536.73,2203762.43,-2441.11
PremiumPercent,1.03,1.81,-0.01
PremiumBp,103.11,181.13,-0.95
ImpliedVolatilityPercent,25.21,25.21,37.84
ImpliedVolatilityBp,105.25,99.02,94.86
DeltaPercent,1.88,-2.71,-0.05
GammaPercent,230.82,288.96,-25.07
